In [1]:
!pip install pysentimiento
!pip install pandas

You should consider upgrading via the '/Users/pzamudio/devel/analisis-libertad-responsable-uy/.venv/bin/python -m pip install --upgrade pip' command.
You should consider upgrading via the '/Users/pzamudio/devel/analisis-libertad-responsable-uy/.venv/bin/python -m pip install --upgrade pip' command.


In [2]:
import pandas as pd
from collections import Counter
from pysentimiento import create_analyzer

/Users/pzamudio/devel/analisis-libertad-responsable-uy/.venv/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Data checks

In [3]:
df = pd.read_csv("data/inputs/tweets_libertad_responsable.csv")
df.head(1).T

,0
id,1395409435347009537
conversation_id,1395409435347009537
created_at,2021-05-20 13:01:49 -03
date,2021-05-20
time,13:01:49
timezone,-300
user_id,921390373
username,icomem_oficial
name,ICOMEM
place,NaN


In [4]:
df.tweet[0]

'Desde ICOMEM hacemos un llamamiento a la responsabilidad. \n\nSon días de disfrutar del fútbol pero nunca sin olvidar el sentido común. \nEstamos cerca de volver a disfrutar del fútbol con toda la pasión pero de momento hagámoslo con cabeza. \n\n#LibertadResponsable #ConCabeza https://t.co/k9Q800L6yn'

# Sentiment Analyzer

In [5]:
# run analyzer
sentiment_analyzer = create_analyzer(task="sentiment", lang="es")
results_sentiment = sentiment_analyzer.predict(df.tweet)

Counter([r.output for r in results_sentiment])

100%|██████████| 18/18 [00:00<00:00, 205.32ba/s]
The following columns in the test set don't have a corresponding argument in `RobertaForSequenceClassification.forward` and have been ignored: text. If text are not expected by `RobertaForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Prediction *****
  Num examples = 548
  Batch size = 32
You're using a PreTrainedTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
100%|██████████| 18/18 [01:23<00:00,  4.66s/it]


Counter({'NEU': 312, 'NEG': 201, 'POS': 35})

# Emotion Analyzer

In [6]:
emotion_analyzer = create_analyzer(task="emotion", lang="es")
results_emotion = emotion_analyzer.predict(df.tweet)

Counter([r.output for r in results_emotion])

loading configuration file config.json from cache at /Users/pzamudio/.cache/huggingface/hub/models--pysentimiento--robertuito-emotion-analysis/snapshots/2a1fb82f525912c23a8187eeea418751049d5056/config.json
Model config RobertaConfig {
  "_name_or_path": "pysentimiento/robertuito-emotion-analysis",
  "architectures": [
    "RobertaForSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "others",
    "1": "joy",
    "2": "sadness",
    "3": "anger",
    "4": "surprise",
    "5": "disgust",
    "6": "fear"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "anger": 3,
    "disgust": 5,
    "fear": 6,
    "joy": 1,
    "others": 0,
    "sadness": 2,
    "surprise": 4
  },
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 130,
  "mode

Counter({'others': 361, 'joy': 69, 'anger': 108, 'sadness': 9, 'fear': 1})

# Save results

In [7]:
results_sentiment[0]

AnalyzerOutput(output=NEU, probas={NEU: 0.769, POS: 0.226, NEG: 0.005})

In [9]:
df_results = df.copy()

df_results['sentiment'] = [r.output for r in results_sentiment]
df_results['sentiment_probs'] = [r.probas for r in results_sentiment]

df_results['emotion'] = [r.output for r in results_emotion]
df_results['emotion_probs'] = [r.probas for r in results_emotion]

df_results[['tweet', 'sentiment', 'emotion']].head(10)


,tweet,sentiment,emotion
0,Desde ICOMEM hacemos un llamamiento a la respo...,NEU,others
1,@LuisLacallePou @MatiasBarreto_ @AlfieIsaac #U...,NEU,others
2,"🌿🇺🇾✨#Uruguay de vacuna 💉,muchas gracias al gob...",NEU,joy
3,Siempre tendremos aquellos con la “viveza crio...,NEG,anger
4,Regresaron a #Uruguay los niños de la calle #l...,NEG,others
5,"Mi ciudad, #Florida , esta conmovida:hoy hubo ...",NEU,sadness
6,Como sanitarios de “segunda” no podemos aproba...,NEG,joy
7,@_nosoyelena Deje de salir señora y no tendrá ...,NEG,others
8,Lo mismo de siempre. La perrada no entiende q...,NEG,anger
9,@UNESCO_es @ONU_es @pnud @opsoms @WHO @JoeBide...,NEU,joy


In [10]:
df_results.to_csv("data/outputs/tweets_libertad_responsable_analysis.csv", index=False)